# Combined figure for CESM104 responses

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import os
import random as rd
from lmfit import Model
from lmfit.model import save_modelresult, load_modelresult
from scipy import optimize
import importlib
import xarray as xr
from data_loading_functions import *

plt.rcParams.update({'figure.max_open_warning': 0})

In [2]:
# Define functions:


def exp_part1(t, S1, tau1):
    return S1*(1 - np.exp(-t/tau1))
def exp_part2(t, S2, tau2):
    return S2*(1 - np.exp(-t/tau2))
def exp_part3(t, S3, tau3):
    return S3*(1 - np.exp(-t/tau3))
def osc_parts(t, Sosc1, Sosc2, taup, Tq):
    p = -1/taup; q = 2*np.pi/Tq
    osc_part1 = Sosc1*(1 - np.exp(-t/taup)*(np.cos(q*t) + p/q*np.sin(q*t)))
    osc_part2 = Sosc2*(1 - np.exp(-t/taup)*(np.cos(q*t) - q/p*np.sin(q*t)))
    return osc_part1 + osc_part2

def twoexp_function(t, S1, S2, tau1, tau2):
    return exp_part1(t, S1, tau1) + exp_part2(t, S2, tau2)
def threeexp_function(t, S1, S2, S3, tau1, tau2, tau3):
    return exp_part1(t, S1, tau1) + exp_part2(t, S2, tau2) + exp_part3(t, S3, tau3)
def expandosc_function(t, S1, S2, Sosc1, Sosc2, tau1, tau2, taup, Tq):
    return exp_part1(t, S1, tau1) + exp_part2(t, S2, tau2) + osc_parts(t, Sosc1, Sosc2, taup, Tq)

exp_model1 = Model(exp_part1); exp_model2 = Model(exp_part2); 
twoexp_model = Model(twoexp_function);
twoexp_model.set_param_hint('S1', value=4, min=0, max=10.0)
twoexp_model.set_param_hint('S2', value=2, min=0, max=10.0)
twoexp_model.set_param_hint('tau1', value=4, min=0, max=8)
twoexp_model.set_param_hint('tau2', value=20, min=8, max=1000)

exp_model3 = Model(exp_part3); threeexp_model = Model(threeexp_function);
threeexp_model.set_param_hint('S1', value=4, min=0, max=10.0)
threeexp_model.set_param_hint('S2', value=2, min=0, max=10.0)
threeexp_model.set_param_hint('S3', value=2, min=0, max=10.0)
threeexp_model.set_param_hint('tau1', value=4, min=0, max=8)
threeexp_model.set_param_hint('tau2', value=20, min=8, max=100)
threeexp_model.set_param_hint('tau3', value=200, min=100, max=1000)

osc_model = Model(osc_parts); expandosc_model = Model(expandosc_function)
expandosc_model.set_param_hint('S1', value=4, min=0, max=10.0)# guess the same as for two-box model 
expandosc_model.set_param_hint('S2', value=2, min=0, max=10.0)
expandosc_model.set_param_hint('Sosc1', value=0.5, min=0, max=4.0)
expandosc_model.set_param_hint('Sosc2', value=0.5, min=0, max=4.0)
expandosc_model.set_param_hint('tau1', value=4, min=0, max=8)
expandosc_model.set_param_hint('tau2', value=20, min=8, max=1000)


expandosc_model.set_param_hint('taup', value=600, min=20, max=1000)
#expandosc_model.set_param_hint('Tq', value=400, min=40, max=2000) # used for longrunmip
expandosc_model.set_param_hint('Tq', value=100, min=40, max=2000)


def exp_part1_fixedpar(t, par_values):
    return exp_part1(t, S1 = par_values['S1'], tau1 = par_values['tau1'])
def exp_part2_fixedpar(t, par_values):
    return exp_part2(t, S2 = par_values['S2'], tau2 = par_values['tau2'])
def exp_part3_fixedpar(t, par_values):
    return exp_part3(t, S3 = par_values['S3'], tau3 = par_values['tau3'])    
    
def osc_parts_fixedpar(t, par_values):
    return osc_parts(t, Sosc1 = par_values['Sosc1'], Sosc2 = par_values['Sosc2'], taup = par_values['taup'], Tq = par_values['Tq'])
def osc_part1_fixedpar(t, par_values):
    return osc_part1(t, Sosc1 = par_values['Sosc1'], taup = par_values['taup'], Tq = par_values['Tq'])
def osc_part2_fixedpar(t, par_values):
    return osc_part2(t, Sosc2 = par_values['Sosc2'], taup = par_values['taup'], Tq = par_values['Tq'])

def twobox_model_fixedpar(t, par_values):
    return exp_part1_fixedpar(t, par_values) + exp_part2_fixedpar(t, par_values)
def threebox_model_fixedpar(t, par_values):
    return exp_part1_fixedpar(t, par_values) + exp_part2_fixedpar(t, par_values) + exp_part3_fixedpar(t, par_values)
def oscillatory_model_fixedpar(t, par_values):
    return exp_part1_fixedpar(t, par_values) + exp_part2_fixedpar(t, par_values) + osc_parts_fixedpar(t, par_values)        


In [3]:
def find_longrunmip_files(model, exp):
    directory = '../../Other_data/longrunmip_data/'
    file_str = model + '_' + exp
    filenames = [f.name for f in os.scandir(directory) if file_str in f.name]
    filenames.sort()
    tas_file = [filename for filename in filenames if 'tas' in filename][0]; 
    nettoa_file = [filename for filename in filenames if 'netTOA' in filename][0];
    return [tas_file, nettoa_file]

def get_tas(model, exp, add_0 = True, remove_nan = False, return_years = False):
    directory = '../../Other_data/longrunmip_data/'
    [tas_file, nettoa_file] = find_longrunmip_files(model, exp)
    ds_tas = xr.open_dataset(directory + tas_file)
    deltaT = ds_tas.tas.values
    if remove_nan == True:
        deltaT = deltaT[np.isnan(deltaT)==False]
    years = np.arange(1,len(deltaT)+1)
    if add_0 == True:
        deltaT = np.concatenate([[0],deltaT])
        years = np.concatenate(([0],years))
        
    if return_years == True:
        return [years, deltaT]
    else:
        return deltaT
    
def modelresult_figure_longrunmip(response_model, model_result, years0, deltaT0, axis = None, length_restriction = None):
    # takes in response models: 'twoexp', 'threeexp', 'expandosc'
    if axis == None:
        #then create new axis:
        fig, axis = plt.subplots(figsize = [8,6])
        
    axis.set_xlabel('Year',fontsize = 18)
    axis.set_ylabel('T [K]',fontsize = 18)
    axis.tick_params(axis='both',labelsize=18)
    axis.plot(years0, deltaT0, color = 'black')
    
    if length_restriction == None:
        model_best_fit = model_result.best_fit
        model_data = model_result.data
    else: # shorten all data. +1 because year 0 is included
        model_best_fit = model_result.best_fit[:length_restriction+1]
        model_data = model_result.data[:length_restriction+1]
        years0 = years0[:length_restriction+1]
     
    axis.set_xlim(min(years0),max(years0))
    residuals = model_best_fit - model_data
    
    model_rmse = np.sqrt(np.mean(residuals**2))
    #model_rmse = np.sqrt(np.mean(model_result.residual**2))
    axis.plot(years0, model_best_fit, '-', label='best fit', color = 'blue', linewidth = 5)
    
    # plot components
    axis.plot(years0, exp_model1.eval(model_result.params, t=years0), color = 'lightblue')
    
    if response_model == 'twoexp':
        axis.plot(years0, exp_model2.eval(model_result.params, t=years0), color = 'lightblue')
        axis.set_title('Two-exponential fit', fontsize = 18)
        partoshow = ['S1', 'S2', 'tau1', 'tau2']
        textlabels = ['$S_1$', '$S_2$', r'$\tau_1$', r'$\tau_2$']
    elif response_model == 'threeexp':
        axis.plot(years0, exp_model2.eval(model_result.params, t=years0), color = 'lightblue')
        axis.plot(years0, exp_model3.eval(model_result.params, t=years0), color = 'lightblue')
        axis.set_title('Three-exponential fit', fontsize = 18)
        partoshow = ['S1', 'S2', 'S3', 'tau1', 'tau2', 'tau3']
        textlabels = ['$S_1$', '$S_2$', '$S_3$', r'$\tau_1$', r'$\tau_2$', r'$\tau_3$']
    elif response_model == 'expandosc':
        axis.plot(years0, exp_model2.eval(model_result.params, t=years0), color = 'lightblue')
        axis.plot(years0, osc_model.eval(model_result.params, t=years0), color = 'lightblue')
        axis.set_title('Two-exponential and oscillatory fit', fontsize = 18)
        partoshow = ['S1', 'S2', 'Sosc1', 'Sosc2', 'tau1', 'tau2', 'taup', 'Tq']
        textlabels = ['$S_1$', '$S_2$', '$S_{osc1}$', '$S_{osc2}$', r'$\tau_1$', r'$\tau_2$', r'$\tau_p$', r'$T_q$']
        
    estimates = [model_result.best_values[par] for par in partoshow]
    axis.text(0.01, 0.95, 'RMSE = ' + str(np.round(model_rmse,3)), transform=axis.transAxes, fontsize = 16)
    #axis.text(0.75,0.55, 'Estimates:', fontsize=14, transform=axis.transAxes)
    xpos_text = 0.87
    axis.text(xpos_text,0.55, 'Estimates:', fontsize=14, transform=axis.transAxes)
    
    for (ind, estimate) in enumerate(estimates):
        axis.text(xpos_text,0.5*(1-ind/8),textlabels[ind] + ' = ' + str('{:.2f}'.format(np.round(estimate,2))), fontsize=14, transform=axis.transAxes)
        #axis.text(0.75,0.5*(1-ind/8),textlabels[ind] + ' = ' + str('{:.2f}'.format(np.round(estimate,2))), fontsize=14, transform=axis.transAxes)
    return axis
    
    
def plot_3fits_longrunmip(years0, deltaT0, model, exp, load_results = False, save_results = False, save_figure = False, length_restriction = None):
    fig, ax = plt.subplots(ncols = 3, figsize = [25,6]);
    fig.suptitle(model + ' responses to ' + exp, fontsize = 22)
    response_models = ['twoexp', 'threeexp', 'expandosc']
    response_functions = [twoexp_function, threeexp_function, expandosc_function]
    
    rmse_values = []
    for (ax_index, axis) in enumerate(ax):
        response_model = response_models[ax_index]; response_function = response_functions[ax_index]
        
        result_file = '../model_results_longrunmip/' + model + '_' + exp + '_' + response_model + '_results.sav'
        if load_results == True:
            model_result = load_modelresult(result_file, funcdefs = {response_model + '_function': response_function})
            #model_rmse = np.sqrt(np.mean(model_result.residual**2))
            # attribute residual seems to be missing
            # but we can compute it manually from other attributes:
            residuals = model_result.best_fit - model_result.data
            model_rmse = np.sqrt(np.mean(residuals**2))
        else: # obtain new results
            if response_model == 'twoexp':
                model_to_fit = twoexp_model
            elif response_model == 'threeexp':
                model_to_fit = threeexp_model
            elif response_model == 'expandosc':
                model_to_fit = expandosc_model
                 
            model_result = model_to_fit.fit(deltaT0, t=years0)
            model_rmse = np.sqrt(np.mean(model_result.residual**2))
            if response_model == 'expandosc':
                twoexp_rmse = rmse_values[0]
                if model_rmse > twoexp_rmse: 
                    i=0;
                    while model_rmse > twoexp_rmse: # if worse than twoexp, then make new fit
                        print(i, model, exp)
                        taup_guess = 10**rd.uniform(np.log10(20), np.log10(1000))
                        Tq_guess = 10**rd.uniform(np.log10(40), np.log10(2000))
                        expandosc_model.set_param_hint('taup', value=taup_guess, min=20, max=1000)
                        expandosc_model.set_param_hint('Tq', value=Tq_guess, min=40, max=2000)
                        model_result = expandosc_model.fit(deltaT0, t=years0)
                        model_rmse = np.sqrt(np.mean(model_result.residual**2))
                        i += 1; 
                    # reset par hints:
                    expandosc_model.set_param_hint('taup', value=600, min=20, max=1000)
                    expandosc_model.set_param_hint('Tq', value=400, min=40, max=2000)
        if save_results == True:
             save_modelresult(model_result, result_file) 
        
        modelresult_figure_longrunmip(response_model, model_result, years0, deltaT0, axis = axis, length_restriction = length_restriction)
        rmse_values.append(model_rmse)
    if save_figure == True:
        plt.savefig('../Figures/LongRunMIP_modelcomparisons/' + model + '_' + exp + '_linresponses_comparison.pdf', format='pdf', dpi=600, bbox_inches="tight")
    return rmse_values
        

In [ ]:
rmse_list = []
for model in ['CESM104']:
#for model in all_abruptexp:
    if model == 'CESM104':
        exp = 'abrupt2x'
        
    [years0, deltaT0] = get_tas(model, exp, add_0 = True, remove_nan = True, return_years = True)
    #rmse_values = plot_3fits(years0, deltaT0, model, exp)
    #rmse_values = plot_3fits(years0, deltaT0, model, exp, load_results = False, save_results = True, save_figure = True)
    rmse_values_CESM104 = plot_3fits_longrunmip(years0, deltaT0, model, exp, load_results = True, save_results = False, save_figure = False)
    #rmse_values_CESM104 = [model, exp] + rmse_values_CESM104
    rmse_list.append(rmse_values_CESM104)
    plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = [30/2,26/4], constrained_layout=True)
#ax = np.concatenate(axes)
#plt.tight_layout()
#plt.subplots_adjust(hspace=0.01)

for model in ['CESM104']:
#for model in member_dict:
    print(model)
    #if model == 'CESM104':
    #    exp = 'abrupt2x'
        
        #rmse_values_CESM104 = rmse_list[0][2:]
    print(rmse_values_CESM104)   
    [years0, deltaT0] = get_tas(model, exp, add_0 = True, remove_nan = True, return_years = True)
    result_file = '../model_results_longrunmip/' + model + '_' + exp + '_expandosc_results.sav'
    model_result = load_modelresult(result_file, funcdefs = {'expandosc_function': expandosc_function})
    ax = modelresult_figure_longrunmip('expandosc', model_result, years0, deltaT0, axis = ax)
    ax.set_title(model + ' ' + exp, fontsize = 16)
    #ax[k].set_title(model + ' ' + exp + ' ' + member, fontsize = 16)
    rmse_change = 100*(rmse_values_CESM104[2] - rmse_values_CESM104[1])/rmse_values_CESM104[1]
    ax.text(0.01, 0.9, '% change: ' + str(np.round(rmse_change,1)), transform=ax.transAxes, fontsize = 16)

        

#plt.savefig('../Figures/longrunmip_2expandosc_results.png', format='png', dpi=600, bbox_inches="tight")        
#plt.savefig('../Figures/longrunmip_2expandosc_results.pdf', format='pdf', dpi=600, bbox_inches="tight")        


## future prediction figure

In [8]:
# load files downloaded from https://github.com/chrisroadmap/ar6/tree/main/data_output/SSPs
erf_ssp126_all = pd.read_csv('../../Other_data/AR6_ERF/ERF_ssp126_1750-2500.csv')
erf_ssp245_all = pd.read_csv('../../Other_data/AR6_ERF/ERF_ssp245_1750-2500.csv')
erf_ssp370_all = pd.read_csv('../../Other_data/AR6_ERF/ERF_ssp370_1750-2500.csv')
erf_ssp585_all = pd.read_csv('../../Other_data/AR6_ERF/ERF_ssp585_1750-2500.csv')
erf_years = erf_ssp126_all['year']
erf_ssp126 = erf_ssp126_all['total']
erf_ssp245 = erf_ssp245_all['total']
erf_ssp370 = erf_ssp370_all['total']
erf_ssp585 = erf_ssp585_all['total']
F4x = 7.799078433107184 # model mean 4xCO2 ERF copied from: https://github.com/chrisroadmap/ar6/blob/main/data_output/cmip6_twolayer_tuning_params.json


In [ ]:
response_models = ['twoexp', 'threeexp', 'expandosc']
response_functions = [twoexp_function, threeexp_function, expandosc_function]
colors = ['green', 'red', 'blue']

max_t1 = 2500    
max_t = len(erf_years)
#filenames = [f.name for f in os.scandir('../model_results') if f.name not in ['.ipynb_checkpoints', '.DS_Store']]

model = 'CESM104'
exp = 'abrupt2x'
        
[years0, deltaT0] = get_tas(model, exp, add_0 = True, remove_nan = True, return_years = True)
result_file = '../model_results_longrunmip/' + model + '_' + exp + '_expandosc_results.sav'

fig, ax = plt.subplots(figsize = [10,4]);

for i in range(len(response_models)):
    response_model = response_models[i]
    response_function = response_functions[i]
    result_file = model + '_' + exp + '_' + response_model + '_results.sav'

    #if result_file in filenames:
    model_result = load_modelresult('../model_results_longrunmip/' + result_file, funcdefs = {response_model + '_function': response_function})
    par_values = model_result.best_values
    
    #ax.set_title(model + ' ' + exp)
    if response_model == 'twoexp':
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp126, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp245, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp370, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp585, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
    elif response_model == 'threeexp':
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp126, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp245, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp370, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp585, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
    elif response_model == 'expandosc':
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp126, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp245, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp370, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp585, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
    ax.grid()
    #ax.set_ylim(-1,4)
    #ax.set_xlim(1750, 2025)

## Combined figure

In [10]:
# load CESM104 AMOC and sea ice data
maxmoc = xr.load_dataset('../../Other_data/longrunmip_data/max_moc_CESM104_abrupt2x.nc')
NHseaice = xr.load_dataset('../../Other_data/longrunmip_data/NH_seaice.nc')
SHseaice = xr.load_dataset('../../Other_data/longrunmip_data/SH_seaice.nc')

In [ ]:
fig = plt.figure(figsize=(18, 6+3+6))
fig.subplots_adjust(hspace=0.26)
gs = gridspec.GridSpec(3, 2, height_ratios=[1, 0.5, 1], width_ratios=[1, 1])

# Top plot 
ax = plt.subplot(gs[0, :])


model = 'CESM104'
#print(model)
#if model == 'CESM104':
exp = 'abrupt2x'
    #rmse_values_CESM104 = rmse_list[0][2:]
print(rmse_values_CESM104)   
[years0, deltaT0] = get_tas(model, exp, add_0 = True, remove_nan = True, return_years = True)
result_file = '../model_results_longrunmip/' + model + '_' + exp + '_expandosc_results.sav'
model_result = load_modelresult(result_file, funcdefs = {'expandosc_function': expandosc_function})
#ax = modelresult_figure('expandosc', model_result, years0, deltaT0, axis = ax, lw = 3)
ax = modelresult_figure_longrunmip('expandosc', model_result, years0, deltaT0, axis = ax)
ax.set_title(model + ' ' + exp, fontsize = 16, fontweight = 'bold')
#ax[k].set_title(model + ' ' + exp + ' ' + member, fontsize = 16)
rmse_change_CESM104 = 100*(rmse_values_CESM104[2] - rmse_values_CESM104[1])/rmse_values_CESM104[1]
ax.text(0.01, 0.9, '% change: ' + str(np.round(rmse_change_CESM104,1)), transform=ax.transAxes, fontsize = 16)
ax.set_xlabel(' ', fontsize = 18)
ax.text(0.01, 1.02, 'a)', transform=ax.transAxes, fontsize = 18, fontweight = 'bold');


# third row  plot
ax = plt.subplot(gs[1, :])
#ax.set_title(model + ' ' + exp + ' AMOC and sea ice', fontsize = 16)
ax.set_ylabel('AMOC (Sv)', fontsize = 18)
ax.set_xlabel('Year', fontsize = 18)
ax.tick_params(axis='both',labelsize=18)
ax.plot(maxmoc.moc, color = 'black')
ax.set_xlim(0,2500)
ax.text(0.01, 1.02, 'b)', transform=ax.transAxes, fontsize = 18, fontweight = 'bold');

# sea ice axis to the right
ax_si = ax.twinx()
ax_si.set_ylabel('Sea-ice area [$10^6$ km$^2$]', fontsize = 18);
ax_si.tick_params(axis='both',labelsize=18)
ax_si.plot(NHseaice.seaicearea.values*10**(-12), '--', color = 'gray')
ax_si.plot(SHseaice.seaicearea.values*10**(-12), ':', color = 'gray')
# use a factor 10^6 to convert from m^2 to km^2, then divide out another factor 10^6 and put this in ylabel instead


# bottom figure
ax = plt.subplot(gs[2, :])
response_models = ['twoexp', 'threeexp', 'expandosc']
response_functions = [twoexp_function, threeexp_function, expandosc_function]
colors = ['green', 'red', 'blue']
  
max_t = len(erf_years)
model = 'CESM104'
exp = 'abrupt2x'
        
[years0, deltaT0] = get_tas(model, exp, add_0 = True, remove_nan = True, return_years = True)
result_file = '../model_results_longrunmip/' + model + '_' + exp + '_expandosc_results.sav'


for i in range(len(response_models)):
    response_model = response_models[i]
    response_function = response_functions[i]
    result_file = model + '_' + exp + '_' + response_model + '_results.sav'

    #if result_file in filenames:
    model_result = load_modelresult('../model_results_longrunmip/' + result_file, funcdefs = {response_model + '_function': response_function})
    par_values = model_result.best_values
    
    #ax.set_title(model + ' ' + exp)
    if response_model == 'twoexp':
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp126, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i], label = 'two-box')
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp245, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp370, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp585, np.diff(twobox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
    elif response_model == 'threeexp':
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp126, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i], label = 'three-box')
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp245, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp370, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp585, np.diff(threebox_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
    elif response_model == 'expandosc':
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp126, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i], label = 'four-box with oscillation')
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp245, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp370, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
        ax.plot(erf_years, 2/F4x*np.convolve(erf_ssp585, np.diff(oscillatory_model_fixedpar(np.arange(0,max_t), par_values = par_values)), mode = 'full')[:max_t], color = colors[i])
ax.set_title('Responses to historical and future scenario forcing', fontsize = 16)
ax.grid()
ax.legend(fontsize = 14)
#ax.set_ylim(-1,4)
#ax.set_xlim(1750, 2025)
ax.set_xlabel('Year',fontsize = 18)
ax.set_ylabel('T [K]',fontsize = 18)
ax.set_xlim(min(erf_years),max(erf_years))
ax.tick_params(axis='both',labelsize=18)
ax.text(0.01, 1.02, 'c)', transform=ax.transAxes, fontsize = 18, fontweight = 'bold');


#plt.savefig('../cesm104_responses_and_pred.pdf', format='pdf', dpi=600, bbox_inches="tight")                
